In [1]:
from dotenv import load_dotenv
import os   
import getpass
from pinecone import Pinecone

load_dotenv(override=True)


True

In [2]:
os.environ["PINECONE_API_KEY"] = os.getenv("PINECONE_API_KEY")

In [15]:
pinecone_api_key = os.environ.get("PINECONE_API_KEY")

In [ ]:
if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("pcsk_4Wtmki_AHQqmDcTr5DqBReZBq3LxWFkCKTAYMcPuEixpMK6WMiXhvTEkBR7ZzoLZHEschT")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_ollama.llms import OllamaLLM
from pinecone import ServerlessSpec
from langchain_pinecone import PineconeVectorStore

d:\ML Projects\Langchain_Rag_Projects\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
#LOADING THE DOCUMENTS
def load_documents(docuement_name):
    loader = PyPDFLoader(docuement_name)
    documents = loader.load()
    return documents

In [5]:
doc = load_documents("fyp_research_paper.pdf")

In [6]:
len(doc)

22

In [7]:
#Dicide the docuemnt to chunks
def doc_splitter(document, chunk_size=1000, chunk_overlap=200):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    docs = text_splitter.split_documents(document)
    return docs

In [8]:
docs = doc_splitter(doc)

In [9]:
len(docs)

93

In [10]:
#Embedding the documents
embeddings = OllamaEmbeddings(model="deepseek-r1:1.5b")

In [11]:
embeddings

OllamaEmbeddings(model='deepseek-r1:1.5b', validate_model_on_init=False, base_url=None, client_kwargs={}, async_client_kwargs={}, sync_client_kwargs={}, mirostat=None, mirostat_eta=None, mirostat_tau=None, num_ctx=None, num_gpu=None, keep_alive=None, num_thread=None, repeat_last_n=None, repeat_penalty=None, temperature=None, stop=None, tfs_z=None, top_k=None, top_p=None)

In [12]:
vectors = embeddings.embed_query("What is the main topic of the research paper?")

In [13]:
len(vectors)

1536

In [ ]:
vectors

In [16]:
pc = Pinecone(api_key=pinecone_api_key)
index_name = "langchainvectors"

index = pc.Index(index_name)

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [17]:
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(docs))]
vector_store.add_documents(documents=docs, ids=uuids)


['973f86f1-e99e-4105-8049-c60d4a3bea91',
 '6071b483-e2fb-402d-b78a-6f98c1c8d440',
 '45ee8d84-93d2-4400-ac50-e49c714285ee',
 '56c41bd7-8059-48fc-aea4-6af346393589',
 'fd83d501-c36b-4bab-9328-4f4a3386020c',
 '3fc849f6-530d-4e96-a2ae-681c065b7698',
 'dfac73f3-7225-4f37-9f97-42231e9e6b43',
 '9190d86b-6856-4745-9d4d-4bc8a6c05141',
 '68a7a96d-55f3-407a-a076-3b196d66ec5e',
 '5a4bab80-8161-49b4-9990-a7d6054645b5',
 'a5685acd-c179-4c00-927b-5f17aaf0025f',
 '266b4f8c-d9ec-4b66-91a4-aa49e236b6c3',
 '679015c5-8231-40b1-a9c4-ac69f3b2db04',
 '03a49c60-4639-457c-9752-578082813257',
 'dc0218be-12e8-4b2f-a3ea-bb26bdbaea60',
 'a380a567-a1ff-48b3-9736-ffc9a084e5cc',
 '6179aade-5ccd-47e5-94a9-8885535e0b9b',
 '06a633d4-03be-4c78-a645-d24e3e819373',
 'c9094249-3061-4618-8536-967ae0c9d491',
 'd30a0d1e-0f21-48e7-9715-86a21ceebfb0',
 '8c140ec3-923c-4e2d-b12d-322531b16607',
 '76b28bd9-b60c-4a2a-b943-aaad5a62c324',
 '2763a3de-b904-43e4-b8a2-0538af168100',
 '91794767-af0b-40bf-8d88-6e4c1c331e92',
 'eba3ce11-5b68-

In [23]:
#cosine similarity retrival
def cosine_similarity_search(query, k=5):
    results = vector_store.similarity_search(
    query=query,
    k=k,
    filter=None,)
    return results

In [20]:
def retival_query(query, k=5):
    retriever = vector_store.as_retriever(
        search_type="similarity_score_threshold",
        search_kwargs={"k": k, "score_threshold": 0.4},
    )
    result = retriever.invoke(query, filter={"source": "news"})
    return result

In [21]:
q = "A Star Algorithm"

In [24]:
cosine_similarity_search(query=q)

[Document(id='84d11a2c-132f-41b3-a861-29a35e1362fd', metadata={'author': 'IJIETAP Assistant Editor', 'creationdate': '2025-08-11T00:42:50+07:00', 'creator': 'Microsoft® Word for Microsoft 365', 'moddate': '2025-08-11T00:42:50+07:00', 'page': 14.0, 'page_label': '15', 'producer': 'Microsoft® Word for Microsoft 365', 'source': 'fyp_research_paper.pdf', 'total_pages': 22.0}, page_content='an optimal balance between overfitting and underfitting. \nFor the selected AdaBoost classifier, a 10 -fold cross-validation scheme with stratified sampling was utilized to ensure \na more robust evaluation and mitigate overfitting. Stratified sampling maintained consistent class proportions across all folds, \npreserving the integrity of the imbalanced dataset. The evaluation results, including the F1 score, accuracy, and AUC -ROC \nscore, were examined to diagnose the model ’s performance. Based on this diagnosis, it was determined that the optimized \nAdaBoost model was not facing overfitting or under